## Uso de libreria de Traduccion:
### https://pypi.org/project/googletrans/

In [ ]:
# Instalando libreria
#!pip3 install googletrans==3.1.0a0

#### Declara librerias

In [1]:
#%%time
from googletrans import Translator, constants
from pprint import pprint
import pandas as pd
import time

#### Inicializa objeto

In [2]:
translator = Translator()

#### Archivo Input

In [3]:
#Leer archivo Csv
name_file = 'movies_Ruben_limpio'
ext_file = '.csv'
name_file_input = name_file + ext_file
df_films = pd.read_csv(name_file_input)

#### Proceso de traducción

In [4]:
#Hora de inicio
hora = time.localtime()
time_string = time.strftime("%m/%d/%Y, %H:%M:%S", hora)
print(time_string)

#Lee dataframe de peliculas, crea dataframe temporal y prepara cadena de texto con las descripciones para la traducción
descripcion = ''
cadena_input  = ''
nro_rows = 0
num_file = 1
ingreso = 0
numreg_es = 0
numreg_en = 0
numreg_tot = 0
df_temporal = pd.DataFrame()
df_temporal['titulo'] = ''
df_temporal['link'] = ''
df_temporal['descripcion'] = ''
df_temporal['idioma'] = ''
df_temporal['traduccion'] = ''
for index, row in df_films.iterrows():
    nro_rows = nro_rows + 1
    numreg_tot = numreg_tot + 1
    xtitulo = row['titulo']
    xlink = row['link']
    xdescripcion = row['descripcion']
    if pd.isna(xdescripcion): #Verifica si no tiene descripción 
        descripcion = 'Without description'
    else:
        descripcion = row['descripcion']
    #Inicializa cadena Input
    if nro_rows == 1:
        cadena_input = ''
    #Arma cadena de Input con las descripciones previa a la traducción    
    cadena_input = cadena_input + "'" + descripcion + "',"
    #Identifica si la descripción está en Inglés(en) o Español(es)
    idioma = str(translator.detect(descripcion))
    idioma = idioma[14:16] #obtiene tipo:en,es
    #Cuenta el número de pelis en inglés y español
    if idioma == 'es':
        numreg_es = numreg_es + 1
    else:
        numreg_en = numreg_en + 1
    #Agrega registros al dataframe temporal para almacenar los registros que se van leyendo
    df_temporal = df_temporal.append({'titulo': xtitulo, 'link': xlink, 'descripcion': descripcion, 'idioma': idioma } , ignore_index=True)
    #Verifica si la longitud de la cadena sea menor a 15,000 caracteres previa a la traducción
    extension_cadena = len(cadena_input)
    if ((extension_cadena > 14000) and (extension_cadena < 15000)):
        ingreso = 1 #Enciende flag de control
        #Extrae ultimo caracter ","
        long_cadena = len(cadena_input) - 1
        cadena_input = cadena_input[0:long_cadena]
        cadena_input = '[' + cadena_input + ']'
        #Llama a función de traducción (máximo 15,000) caracteres
        translation = translator.translate(cadena_input, dest='es')
        result_transl = translation.text
        long_cadena = len(result_transl) - 1
        cadena_output = result_transl[1:long_cadena] #Quita los corchetes []
        #separa descripciones traducidas para agregarlas al dataframe de traducción
        cadena_tmp=''
        anterior_caracter=''
        df_traduce = pd.DataFrame()
        df_traduce['traduce'] = ''
        for i in cadena_output:
            if i == "," and anterior_caracter== "'": #Pregunta por subcadena "',"
                df_traduce = df_traduce.append({'traduce' : cadena_tmp} , ignore_index=True)
                cadena_tmp=''
            else:
                cadena_tmp = cadena_tmp + i
            anterior_caracter= i #Guarda caracter actual

        df_traduce = df_traduce.append({'traduce' : cadena_tmp} , ignore_index=True)
        #Actualiza columna de Traducción del dataframe temporal con la traducción
        df_temporal['traduccion'] = df_traduce['traduce']
        #Exporta dataframe con la traducción a csv
        name_file_output = name_file + '_traducido_' + str(num_file) + ext_file 
        df_temporal.to_csv(name_file_output, encoding='utf-8-sig')
        num_file = num_file + 1
        #Inicializa valores
        nro_rows = 0
        df_temporal = pd.DataFrame()
        df_temporal['titulo'] = ''
        df_temporal['link'] = ''
        df_temporal['descripcion'] = ''
        df_temporal['idioma'] = ''
        df_temporal['traduccion'] = ''
        print('Generando archivo traducido: ' + name_file_output)

#Traduce y genera archivo csv traducido para aquellas archivos input que tienen menos de 15,000 caracteres
if (ingreso == 0) or (num_file >= 2):
    #Extrae ultimo caracter ","
    long_cadena = len(cadena_input) - 1
    cadena_input = cadena_input[0:long_cadena]
    cadena_input = '[' + cadena_input + ']'
    #Llama a función de traducción (máximo 15,000) caracteres
    translation = translator.translate(cadena_input, dest='es')
    result_transl = translation.text
    long_cadena = len(result_transl) - 1
    cadena_output = result_transl[1:long_cadena] #Quita los corchetes []
    #separa descripciones traducidas para agregarlas al dataframe de traducción
    cadena_tmp=''
    anterior_caracter=''
    df_traduce = pd.DataFrame()
    df_traduce['traduce'] = ''
    for i in cadena_output:
        if i == "," and anterior_caracter== "'": #Pregunta subcadena "',"
            df_traduce = df_traduce.append({'traduce' : cadena_tmp} , ignore_index=True)
            cadena_tmp=''
        else:
            cadena_tmp = cadena_tmp + i
        anterior_caracter= i

    df_traduce = df_traduce.append({'traduce' : cadena_tmp} , ignore_index=True)    
    #Actualiza columna de Traducción del dataframe temporal con la traducción
    df_temporal['traduccion'] = df_traduce['traduce']
    #Exporta dataframe con la traducción a csv
    name_file_output = name_file + '_traducido_' + str(num_file) + ext_file 
    df_temporal.to_csv(name_file_output, encoding='utf-8-sig')
    print('Generando archivo traducido: ' + name_file_output)
    print('')
    print('*****************************************')
    print('***** Fin de proceso de Traducción ******')
    print('*****************************************')
    print('')
    print('Número de registros en inglés : ' + str(numreg_en))
    print('Número de registros en español: ' + str(numreg_es))
    print('Número total de registros     : ' + str(numreg_tot))

#Hora fin
hora = time.localtime()
time_string = time.strftime("%m/%d/%Y, %H:%M:%S", hora)
print(time_string)

05/06/2022, 09:40:47
Generando archivo traducido: movies_Ruben_limpio_M_traducido_1.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_2.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_3.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_4.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_5.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_6.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_7.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_8.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_9.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_10.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_11.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_12.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_13.csv
Generando archivo traducido: movies_Ruben_limpio_M_traducido_14.csv
Generando archivo traducido: movies_